In [ ]:
!pip install -q flwr[simulation] flwr-datasets[vision] datasets

In [ ]:
from collections import OrderedDict, defaultdict
from datasets import DatasetDict, Dataset
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
disable_progress_bar()


Training on cuda
Flower 1.14.0 / PyTorch 2.5.1+cu121


In [ ]:
def normalize_dataset(dataset_dict):
    """
    Normalize all numerical features in each split of the DatasetDict while retaining the original structure.

    Args:
        dataset_dict (DatasetDict): The dataset with splits (e.g., train, test).

    Returns:
        DatasetDict: A new DatasetDict with normalized features.
    """

    def normalize_split(split_dataset):
        feature_keys = [key for key in split_dataset.column_names if key != "is_spam"]  # Exclude the label
        features = np.array([split_dataset[key] for key in feature_keys]).T  # Extract features as a matrix
        labels = split_dataset["is_spam"]  # Extract labels

        # Apply Min-Max Scaling
        scaler = MinMaxScaler()
        normalized_features = scaler.fit_transform(features)

        # Create a dictionary for normalized data
        normalized_data = {key: normalized_features[:, idx] for idx, key in enumerate(feature_keys)}
        normalized_data["is_spam"] = labels  # Retain the labels

        # Convert back to a Dataset
        return split_dataset.from_dict(normalized_data)

    # Normalize each split (e.g., train, validation, test)
    normalized_splits = {split_name: normalize_split(split) for split_name, split in dataset_dict.items()}

    # Return the new normalized DatasetDict
    return normalized_splits

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:

global_partitions = {}

def preload_datasets():
    """Preload and partition the dataset into global_partitions."""
    global global_partitions
    fds = FederatedDataset(dataset="mstz/spambase", partitioners={"train":configs["NUM_CLIENTS"]}, preprocessor=normalize_dataset)
    for partition_id in range(configs["NUM_CLIENTS"]):
        partition = fds.load_partition(partition_id)
        train_test_split = partition.train_test_split(test_size=configs["test_size"], seed=configs["seed"])
        train_val_split = train_test_split["train"].train_test_split(test_size=configs["test_size"], seed=configs["seed"])

        global_partitions[partition_id] = {
            "train": DataLoader(train_val_split["train"], batch_size=configs["BATCH_SIZE"], shuffle=True),
            "val": DataLoader(train_val_split["test"], batch_size=configs["BATCH_SIZE"]),
            "test": DataLoader(train_test_split["test"], batch_size=configs["BATCH_SIZE"]),
        }

#Non-IID data. DirichletPartitioner

In [ ]:
class SoftDecisionTree(nn.Module):
    def __init__(self, input_size, depth=3, output_size=2, temperature=1.0,feature_subset=None):
        super(SoftDecisionTree, self).__init__()
        self.depth = depth
        self.num_nodes = 2 ** depth - 1  # Number of decision nodes
        self.num_leaves = 2 ** depth    # Number of leaf nodes
        self.temperature = temperature
        self.feature_subset = feature_subset


        actual_input_size = len(feature_subset) if feature_subset else input_size

        self.dropout = nn.Dropout(p=0.2)
        # Decision nodes (cut points for splits)
        self.decision_nodes = nn.ModuleList([
            nn.Linear(actual_input_size, 1) for _ in range(self.num_nodes)
        ])

        # Leaf scores (probabilities for classification)
        self.leaf_scores = nn.Linear(self.num_leaves, output_size)

    def forward(self, x):

        if self.feature_subset:
            x = x[:, self.feature_subset]

        x = self.dropout(x)
        # Compute probabilities for each decision node
        decision_probs = torch.cat([torch.sigmoid(node(x)) for node in self.decision_nodes], dim=1)

        # Compute leaf probabilities
        leaf_probs = self.compute_leaf_probs(decision_probs)

        # Compute class probabilities
        output = self.leaf_scores(leaf_probs)


        #return F.log_softmax(output, dim=1)
        return output


    def compute_leaf_probs(self, decision_probs):
      """
      Compute the probabilities for each leaf node using the decision probabilities.
      """
      batch_size = decision_probs.size(0)
      leaf_probs = torch.ones(batch_size, self.num_leaves, device=decision_probs.device)

      for depth in range(self.depth):
          stride = 2 ** (self.depth - depth - 1)
          for leaf_idx in range(0, self.num_leaves, stride * 2):
              node_idx = (2 ** depth - 1) + (leaf_idx // (stride * 2))

              # Ensure alignment of shapes
              decision_prob = decision_probs[:, node_idx].unsqueeze(1)  # Shape [batch_size, 1]
              complement_prob = (1 - decision_prob)  # Shape [batch_size, 1]

              # Update probabilities
              leaf_probs[:, leaf_idx:leaf_idx + stride] *= decision_prob
              leaf_probs[:, leaf_idx + stride:leaf_idx + stride * 2] *= complement_prob

      return leaf_probs



In [ ]:
class SoftRandomForest(nn.Module):
    def __init__(self, num_trees, input_size, depth=3, output_size=2, temperature=1.0, num_features=None):
        super(SoftRandomForest, self).__init__()
        self.num_trees = num_trees
        self.trees = nn.ModuleList([
            SoftDecisionTree(
                input_size=input_size,
                depth=depth,
                output_size=output_size,
                temperature=temperature,
                feature_subset=torch.randperm(input_size)[:num_features].tolist() if num_features else None
            ) for _ in range(num_trees)
        ])

    def forward(self, x):
        # Collect predictions from all trees
        tree_outputs = torch.stack([tree(x) for tree in self.trees], dim=0)  # Shape: [num_trees, batch_size, output_size]

        # Average probabilities for classification
        avg_output = torch.mean(tree_outputs, dim=0)  # Shape: [batch_size, output_size]
        return avg_output


In [ ]:
def train_forest(forest, trainloader, epochs, lr):
    """Train the Soft Random Forest."""

    criterion = nn.CrossEntropyLoss()
    #criterion = nn.NLLLoss()

    optimizers = [
        torch.optim.Adam(tree.parameters(), lr=lr, weight_decay=1e-4) for tree in forest.trees
    ]

    schedulers = [
        torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
        for optimizer in optimizers
    ]

    all_inputs = []
    all_labels = []
    for batch in trainloader:
        inputs = torch.stack([value for key, value in batch.items() if key != "is_spam"], dim=1).float()
        labels = batch["is_spam"].long()
        all_inputs.append(inputs)
        all_labels.append(labels)

    all_inputs = torch.cat(all_inputs, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

    tree_data = []
    for tree_idx in range(len(forest.trees)):
        indices = torch.randint(0, len(all_inputs), (len(all_inputs),))  # Bootstrap sampling
        inputs_bootstrap = all_inputs[indices]
        labels_bootstrap = all_labels[indices]
        inputs_bootstrap = (inputs_bootstrap - inputs_bootstrap.mean(dim=0)) / (inputs_bootstrap.std(dim=0) + 1e-8)#Normalize
        if tree_idx == 0:
          print(f"[DEBUG] Labels bootstrap sample: {labels_bootstrap[:10]}")

        # Store data for the tree
        tree_data.append((inputs_bootstrap, labels_bootstrap))


    best_losses = [float("inf")] * len(forest.trees)
    patience_counters = [0] * len(forest.trees)

    for epoch in range(epochs):
        total_loss, correct, total = 0.0, 0, 0

        # Train each tree independently
        for tree_idx, (tree, optimizer, scheduler) in enumerate(zip(forest.trees, optimizers,schedulers)):
            tree.train()



            inputs_bootstrap, labels_bootstrap = tree_data[tree_idx]
            epoch_loss = 0.0

            for batch_idx, batch in enumerate(trainloader):
                if epoch == 0 and tree_idx == 0 and batch_idx == 0:
                  print(f"[DEBUG] First decision node weights (Tree 0): {tree.decision_nodes[0].weight.data}")
                  print(f"[DEBUG] Inputs mean: {inputs_bootstrap.mean():.4f}, std: {inputs_bootstrap.std():.4f}")

                # Zero gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = tree(inputs_bootstrap)
                if epoch == 0 and tree_idx == 0 and batch_idx == 0:
                  print(f"[DEBUG] Outputs sample logits: {outputs[:5]}")

                loss = criterion(outputs, labels_bootstrap)

                # Backward pass and optimization
                loss.backward()
                optimizer.step()

                # Track metrics
                epoch_loss += loss.item()
                total_loss += loss.item()
                _, predicted = torch.max(outputs, dim=1)
                total += labels_bootstrap.size(0)
                correct += (predicted == labels_bootstrap).sum().item()

            #Scheduler step
            avg_epoch_loss = (epoch_loss / len(trainloader))
            scheduler.step()
            #if True:
              #print(f"[DEBUG] Learning rate after scheduler step (Tree {tree_idx}): {optimizer.param_groups[0]['lr']}")

            # Early Stopping Logic
            min_delta = configs["min_delta"]
            if total_loss < best_losses[tree_idx] - min_delta:
                best_losses[tree_idx] = total_loss
                patience_counters[tree_idx] = 0
            else:
                patience_counters[tree_idx] += 1

            if patience_counters[tree_idx] >= configs["patience"]:
                print(f"Tree {tree_idx}: Early stopping at epoch {epoch + 1}")
                break


        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {total_loss:.4f}, Accuracy: {100 * correct / total:.2f}%")


def test_forest(forest, testloader):
    """Evaluate the Soft Random Forest on the test set."""
    #criterion = nn.NLLLoss()
    criterion = nn.CrossEntropyLoss()
    forest.eval()
    total_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for batch in testloader:
            # Extract features and labels
            inputs = torch.stack([value for key, value in batch.items() if key != "is_spam"], dim=1).float()
            labels = batch["is_spam"].long()

            # Get predictions from the forest
            outputs = forest(inputs)
            loss = criterion(outputs, labels)

            # Track metrics
            total_loss += loss.item()
            _, predicted = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return total_loss / len(testloader.dataset), accuracy


In [ ]:
class FederatedClient(NumPyClient):
    def __init__(self, model, trainloader, testloader):
        self.model = model
        self.trainloader = trainloader
        self.testloader = testloader

    def get_parameters(self, config=None):
        """Get model parameters as a list of NumPy arrays."""
        params = [param.cpu().detach().numpy() for param in self.model.parameters()]
        #print(f"[DEBUG] Parameters being sent back to the server: {[p.shape for p in params]}")
        return params
    def set_parameters(self, parameters):
        """Set model parameters from a list of NumPy arrays."""
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        self.model.load_state_dict(state_dict, strict=True)
        #print(f"[DEBUG] Parameters received from the server set successfully.")

    def fit(self, parameters, config):
        """Train the model on the local dataset."""
        #print(f"[DEBUG] Starting training on client with {len(self.trainloader.dataset)} samples.")
        self.set_parameters(parameters)
        lr = config.get("lr", configs["lr"])
        local_epochs = config.get("local_epochs", configs["local_epochs"])

        train_forest(self.model, self.trainloader, epochs=local_epochs, lr=lr)
        return self.get_parameters(), len(self.trainloader.dataset), {}

    def evaluate(self, parameters, config):
        """Evaluate the model on the local test set."""
        self.set_parameters(parameters)
        loss, accuracy = test_forest(self.model, self.testloader)
        #print(f"[DEBUG] Evaluation results - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
        return float(loss), len(self.testloader.dataset), {"accuracy": float(accuracy)}


In [ ]:
def weighted_average(metrics):
    """Aggregate metrics from multiple clients."""
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    total_examples = [num_examples for num_examples, _ in metrics]
    return {"accuracy": sum(accuracies) / sum(total_examples)}

def server_fn(context):
    def on_fit_config_fn(round_num):
      #print(f"[DEBUG] Starting Round {round_num} with configuration.")
      return {"lr": configs["lr"], "local_epochs": configs["local_epochs"]}


    """Server configuration for federated learning."""
    strategy = FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=configs["NUM_CLIENTS"],
        min_evaluate_clients=1,
        min_available_clients=configs["NUM_CLIENTS"],
        on_fit_config_fn=on_fit_config_fn,
        evaluate_metrics_aggregation_fn=weighted_average,
    )
    return flwr.server.ServerAppComponents(strategy=strategy, config=ServerConfig(num_rounds=configs["num_rounds"]))
#Get models from clients and merge the models. Then evaluate the global model.
#Use sklearn to define the models. Probnably dont use flower. Use flower for partitoning. Code is custom mainly.
#Sort and select for merging.
#Random forrest has diffrent depths and shapes. So how do you merge it?
#Not all clients participate in training but they all get the same model
#How do we push the model?
#Eval data set is used to sort. To determine which trees are good.
#MDT, sort and select, no merging you select the trees to form a forrest.

In [ ]:
global configs
configs = {
    "lr": 0.01,
    "local_epochs": 10,
    "NUM_CLIENTS": 3,
    "BATCH_SIZE": 128,
    "num_trees": 50,
    "input_size": 57,
    "depth": 4,
    "output_size": 2,
    "temperature": 1.0,
    "num_features": 25,
    "num_rounds": 8,
    "seed": 42,
    "test_size": 0.2,
    "patience": 5,
    "min_delta": 1e-4,
}



def client_fn(context):
    # Initialize client-specific model and data
    partition_id = context.node_config["partition-id"]
    client_data = global_partitions[partition_id]  # Access preloaded partitions

    trainloader = client_data["train"]
    valloader = client_data["val"]
    testloader = client_data["test"]

    #print(f"[DEBUG] Client {partition_id}: Train size = {len(trainloader.dataset)}, "
    #      f"Val size = {len(valloader.dataset)}, Test size = {len(testloader.dataset)}")


    model = SoftRandomForest(
        num_trees=configs["num_trees"],
        input_size=configs["input_size"],
        depth=configs["depth"],
        output_size=configs["output_size"],
        temperature=configs["temperature"],
        num_features=configs["num_features"])

    # Return the FederatedClient wrapped as a Client instance
    return FederatedClient(model, trainloader, testloader).to_client()



preload_datasets()

# Configure resources for simulation
backend_config = {"client_resources": {"num_cpus": 1}}

if torch.cuda.is_available():
    backend_config["client_resources"]["num_gpus"] = 1.0

# Run simulation
run_simulation(
    server_app=ServerApp(server_fn=server_fn),
    client_app=ClientApp(client_fn=client_fn),
    num_supernodes=configs["NUM_CLIENTS"],
    backend_config=backend_config,
)


/usr/local/lib/python3.11/dist-packages/flwr_datasets/utils.py:109: UserWarning: The currently tested dataset are ['mnist', 'ylecun/mnist', 'cifar10', 'uoft-cs/cifar10', 'fashion_mnist', 'zalando-datasets/fashion_mnist', 'sasha/dog-food', 'zh-plus/tiny-imagenet', 'scikit-learn/adult-census-income', 'cifar100', 'uoft-cs/cifar100', 'svhn', 'ufldl-stanford/svhn', 'sentiment140', 'stanfordnlp/sentiment140', 'speech_commands', 'LIUM/tedlium', 'flwrlabs/femnist', 'flwrlabs/ucf101', 'flwrlabs/ambient-acoustic-context', 'jlh/uci-mushrooms', 'Mike0307/MNIST-M', 'flwrlabs/usps', 'scikit-learn/iris', 'flwrlabs/pacs', 'flwrlabs/cinic10', 'flwrlabs/caltech101', 'flwrlabs/office-home', 'flwrlabs/fed-isic2019']. Given: mstz/spambase.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/se

(ClientAppActor pid=8245) [DEBUG] Labels bootstrap sample: tensor([0, 0, 1, 0, 0, 1, 1, 0, 1, 0])
(ClientAppActor pid=8245) [DEBUG] First decision node weights (Tree 0): tensor([[-0.1452, -0.0961,  0.1771,  0.1246,  0.1010, -0.0358,  0.0051, -0.0428,
(ClientAppActor pid=8245)           0.0541, -0.1640,  0.1233, -0.1680, -0.0419,  0.1033, -0.0361,  0.0948,
(ClientAppActor pid=8245)          -0.0921, -0.1659,  0.1253, -0.0189, -0.1496,  0.0902, -0.1132, -0.1106,
(ClientAppActor pid=8245)           0.1105]])
(ClientAppActor pid=8245) [DEBUG] Inputs mean: 0.0000, std: 0.9995
(ClientAppActor pid=8245) [DEBUG] Outputs sample logits: tensor([[-0.2378, -0.2366],
(ClientAppActor pid=8245)         [-0.2463, -0.2481],
(ClientAppActor pid=8245)         [-0.2212, -0.2747],
(ClientAppActor pid=8245)         [-0.2394, -0.2773],
(ClientAppActor pid=8245)         [-0.2132, -0.2730]], grad_fn=<SliceBackward0>)
(ClientAppActor pid=8244) [DEBUG] Labels bootstrap sample: tensor([1, 0, 0, 0, 0, 1, 1, 0, 0, 

INFO :      aggregate_fit: received 3 results and 0 failures


(ClientAppActor pid=8245) Epoch [10/10], Loss: 189.6306, Accuracy: 82.95%


INFO :      configure_evaluate: strategy sampled 1 clients (out of 3)
INFO :      aggregate_evaluate: received 1 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)


...

INFO :      configure_evaluate: strategy sampled 1 clients (out of 3)
INFO :      aggregate_evaluate: received 1 results and 0 failures
INFO :      
INFO :      [ROUND 8]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)


(ClientAppActor pid=8244) [DEBUG] Labels bootstrap sample: tensor([0, 1, 0, 0, 1, 1, 1, 0, 0, 0])
(ClientAppActor pid=8244) [DEBUG] First decision node weights (Tree 0): tensor([[-0.2095,  0.2388,  0.2479, -0.0392,  0.1922,  0.3199, -0.0804, -0.1976,
(ClientAppActor pid=8244)           0.0455, -0.4851, -0.0920, -0.1103,  0.0556,  0.2425, -0.1263, -0.0336,
(ClientAppActor pid=8244)           0.0316,  0.3196,  0.1895, -0.0557, -0.2927, -0.0998, -0.1748, -0.2053,
(ClientAppActor pid=8244)           0.2644]])
(ClientAppActor pid=8244) [DEBUG] Inputs mean: 0.0000, std: 0.9995
(ClientAppActor pid=8244) [DEBUG] Outputs sample logits: tensor([[ 1.2171, -1.7917],
(ClientAppActor pid=8244)         [ 0.6897, -1.2601],
(ClientAppActor pid=8244)         [ 0.4619, -0.9890],
(ClientAppActor pid=8244)         [-0.3648, -0.1391],
(ClientAppActor pid=8244)         [ 0.1760, -0.6964]], grad_fn=<SliceBackward0>)
(ClientAppActor pid=8245) [DEBUG] Outputs sample logits: tensor([[-0.1720, -0.3381],
(ClientAp

INFO :      aggregate_fit: received 3 results and 0 failures


(ClientAppActor pid=8244) Epoch [10/10], Loss: 139.3594, Accuracy: 85.85%


INFO :      configure_evaluate: strategy sampled 1 clients (out of 3)
INFO :      aggregate_evaluate: received 1 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 8 round(s) in 3795.76s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.006508528797944904
INFO :      		round 2: 0.006493122259258059
INFO :      		round 3: 0.00649926406164511
INFO :      		round 4: 0.006374394078208103
INFO :      		round 5: 0.006506940635097143
INFO :      		round 6: 0.006453407898011347
INFO :      		round 7: 0.006484630441820971
INFO :      		round 8: 0.00655754051301689
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'accuracy': [(1, 0.6188925081433225),
INFO :      	              (2, 0.6188925081433225),
INFO :      	              (3, 0.6188925081433225),
INFO :      	              (4, 0.6449511400651465),
INFO :      	              (5, 0.6188925081433225),
INFO :      	              (6, 0.6449511400651465),
INFO :      	         